In [1]:
import sys
import os
sys.path.append(os.path.abspath('./aima-python'))
#import utils
from games import *
import random

#random.seed(109)

import copy
import itertools
from collections import namedtuple

import numpy as np

from utils import vector_add
import time

In [2]:
class Mancala:
    def __init__(self, pits_per_player=3, stones_per_pit = 2):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        # write your code here

        # to store pit numbers
        valid_pits = []

        # find which pit range belongs to current player
        if self.current_player == 1:
            start, end = self.p1_pits_index
        else:
            start, end = self.p2_pits_index
    
        # get all valid moves
        pit_number = 1
        
        for i in range(start, end + 1):
            if self.board[i] > 0:
                valid_pits.append(pit_number)
            pit_number += 1
    
        return valid_pits
        
        pass
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        # write your code here

        valid_pits = self.valid_move()

        if not valid_pits:
            return None
    
        return random.choice(valid_pits)
    
        pass

    def index_accesor(self, player, pit):
        
        if player == 1:
            return self.p1_pits_index[0] + (pit - 1)
        else:
            return self.p2_pits_index[0] + (pit - 1)

    def get_current_player_mancala(self, player):
        
        if player == 1:
            return self.p1_mancala_index
        else:
            return self.p2_mancala_index

    def get_opposite_pit_index(self, idx):
    
        # ignore the mancalas
        if idx == self.p1_mancala_index or idx == self.p2_mancala_index:
            return None

        #get opposite index
        opp = (len(self.board) - 2) - idx
    
        # check again for mancalas
        if opp == self.p1_mancala_index or opp == self.p2_mancala_index:
            return None
    
        return opp


    def move(self, player, pit):

        # convert to 0-based indexing
        index =  self.index_accesor(player, pit)
        
        # get number of stones at a certain pit
        stones = self.board[index]

        # check valid move
        if not self.valid_move():
            print("INVALID MOVE")
            return None

        #pick stones up
        self.board[index] = 0

        # get opponent's mancala, so we don't put stones in their mancala
        if player == 1:
            opp_mancala = self.p2_mancala_index
        else:
            opp_mancala = self.p1_mancala_index

        current_index = index

        #disperse stones
        
        while stones > 0:
            #wrap around board
            current_index = (current_index + 1) % len(self.board)

            # don't put stones in opponent's pits
            if current_index == opp_mancala:
                continue

            self.board[current_index] += 1
            stones -= 1

        
        return current_index
            
        
    
    def play(self, pit, print_move = False):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        # write your code here

        #check if move is valid in the list of valid moves
        valid_pits = self.valid_move()

        if pit not in valid_pits:
            if not print_move:
                print("INVALID MOVE")
                return self.board
    
        # Record move in self.moves
        self.moves.append((self.current_player, pit))
        if not print_move:
            print(f"Player {self.current_player} chose pit: {pit}")
        
        
        # get current player, current player's mancala, and their opponent
        
        player = self.current_player
        current_player_mancala = self.get_current_player_mancala(player)

        if player ==  1:
            opponent = 2
        else:
            opponent = 1


        # use the function move to perform sowing

        player_move = self.move(player, pit)
        
        if player_move is None:
            return self.board

        
        # Implement Rule: If the last stone lands in an empty pit on your side of the board, capture this
        # stone and any stones in your opponent's pit on the other side of the board,
        # collect all of these stones, including the one that just landed, and place them into your mancala.

        if player_move != current_player_mancala:
        
            # check which side last stone landed on
            if ((player == 1 and self.p1_pits_index[0] <= player_move <= self.p1_pits_index[1]) or
                (player == 2 and self.p2_pits_index[0] <= player_move <= self.p2_pits_index[1])):
        
                # If the last stone lands in an empty pit on your side, capture it and the opposite pit
                # Just after sowing, that pit now has exactly 1 stone
                if self.board[player_move] == 1:
                    opp_index = self.get_opposite_pit_index(player_move)
                    
                    if opp_index is not None and self.board[opp_index] > 0:
                        # capture your 1 stone and all your opponent's stones
                        self.board[current_player_mancala] += self.board[opp_index] + 1
                        self.board[player_move] = 0
                        self.board[opp_index] = 0

        
        
        if self.winning_eval():
            if not print_move:
                print("GAME OVER")
                return self.board
            
     
        # switch players every turn
        self.current_player = (player % 2) + 1

        #opponent_mancala = self.get_current_player_mancala(opponent)
        
        return self.board
    
    def winning_eval(self):
        # check if all of player 1's pits are empty
        p1_empty = all(self.board[i] == 0 
                       for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
    
        # check if all of player 2's pits are empty
        p2_empty = all(self.board[i] == 0 
                       for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
    
        if not (p1_empty or p2_empty):
            return False
    
        # put all of the remaining stones of the loser in their mancala
        
        if p1_empty:
            # add all Player 2's stones into their mancala
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                self.board[self.p2_mancala_index] += self.board[i]
                self.board[i] = 0
    
        if p2_empty:
            # add all Player 1's stones into their mancala
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                self.board[self.p1_mancala_index] += self.board[i]
                self.board[i] = 0
    
        return True  

#AIMA functions filled in

    def actions(self, state):
        # """Return a list of the allowable moves at this point."""
        return state.moves

    def result(self, state, move):
        #"""Return the state that results from making a move from a state."""

        # create a new game, so minmax explores future states 
        new_game = Mancala(self.pits_per_player, self.stones_per_pit)
        # convert to list so it's mutable
        new_game.board = list(state.board)
        # get whose turn it is
        new_game.current_player = state.to_move
    
        # actually move
        new_game.play(move, print_move = True)
    
        # Return a new GameState based on *new_game*, not self
        return GameState(
            to_move = new_game.current_player, # sets next player (to know when to do min vs max)
            utility = None,   # don't compute utility yet                   
            board = tuple(new_game.board),    
            moves = new_game.valid_move()) #get updated valid moves

    def utility(self, state, player):
        # create Mancala object to examin board
        temp = Mancala(self.pits_per_player, self.stones_per_pit)
        temp.board = list(state.board)
        temp.current_player = state.to_move
    
        # get players' mancalas
        p1_mancala = temp.p1_mancala_index
        p2_mancala = temp.p2_mancala_index

        # suggested utility
        if player == 1:
            return temp.board[p1_mancala] - temp.board[p2_mancala]
        else:
            return temp.board[p2_mancala] - temp.board[p1_mancala]

    def terminal_test(self, state):
        # """Return True if this is a final state for the game."""
        temp = Mancala(self.pits_per_player, self.stones_per_pit)
        temp.board = list(state.board)
        temp.current_player = state.to_move
        return temp.winning_eval()

    def to_move(self, state):
        #"""Return the player whose move it is in this state."""
        return state.to_move
            
            
        pass


In [3]:
#minmax function

# from AIMA with new added depth param
def minmax_decision(state, game, depth = 5):
    """Depth-limited Minimax decision.
       Returns the best move for the player whose turn it is in `state`.
    """

    player = game.to_move(state)

    def max_value(s, d):
        if d == depth or game.terminal_test(s):
            return game.utility(s, player)
        v = -np.inf
        for a in game.actions(s):
            v = max(v, min_value(game.result(s, a), d + 1))
        return v

    def min_value(s, d):
        if d == depth or game.terminal_test(s):
            return game.utility(s, player)
        v = np.inf
        for a in game.actions(s):
            v = min(v, max_value(game.result(s, a), d + 1))
        return v

    # Choose the action that maximizes the minimizing player's response
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a), 1))


In [4]:
def play_game_minmax():
    
    game = Mancala(6, 4)

    turns = 0
    player1_turns = 0
    player2_turns = 0

    while True:

        if game.winning_eval():
            break

        # decide which move to make
        if game.current_player == 1:  # AI agent
            # build a GameState from the *current* game
            state = GameState(
                to_move=game.current_player,
                utility=None,                    
                board=tuple(game.board),         
                moves=game.valid_move())
            
            pit = minmax_decision(state, game) # min max AI
            
        else:                               
            pit = game.random_move_generator() # random player
        
        # if no moves are possible, evaluate the winner and stop
        if pit is None:
            game.winning_eval()
            break

    
        # play the move on the real game
        game.play(pit)
        turns += 1
        
        if game.current_player == 1:
            player1_turns += 1
        else:
            player2_turns += 1

     

    

    print("\nList of valid moves:")
    for move in game.moves:
        player, pit = move
        print(f"Player {player} selected pit {pit}")

    return game, turns, player1_turns, player2_turns

In [5]:
#alpha-beta
# From AIMA 
def alpha_beta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


def alpha_beta_cutoff_search(state, game, d=5, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state: game.utility(state, player))
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

In [6]:
def play_game_alphabeta():
    
    game = Mancala(6, 4)

    turns = 0
    player1_turns = 0
    player2_turns = 0


    while True:

        if game.winning_eval():
            break

        # decide which move to make
        if game.current_player == 1:  
            state = GameState(
                to_move=game.current_player,
                utility=0,                    
                board=tuple(game.board),   
                moves=game.valid_move()          
            )
            pit = alpha_beta_cutoff_search(state, game, d=4)
        else:                                   # 
            pit = game.random_move_generator()
        
        # if no moves are possible, evaluate winner and stop
        if pit is None:
            game.winning_eval()
            break


        # play the move on the real game
        game.play(pit)
        turns += 1

        turns += 1
        
        if game.current_player == 1:
            player1_turns += 1
        else:
            player2_turns += 1

     


    print("\nList of valid moves:")
    for move in game.moves:
        player, pit = move
        print(f"Player {player} selected pit {pit}")

    return game, turns, player1_turns, player2_turns

In [7]:
# Random vs Random (100 iterations)

In [8]:
def play_game():
    
    game = Mancala(6, 4)
    #game.display_board()

    turns = 0
    player1_turns = 0
    player2_turns = 0
    

    while True:

        if game.winning_eval():
            stop = True
            break
            
        #generate a random valid pit play
        pit = game.random_move_generator()

        #if no moves are possible, then evaluate the winner
        if pit is None:
            game.winning_eval()
            break

        # play the actual move
        game.play(pit)
        #game.display_board()

        turns += 1
        
        if game.current_player == 1:
            player1_turns += 1
        else:
            player2_turns += 1

     

    

    print("\nList of valid moves:")
    for move in game.moves:
        player, pit = move
        print(f"Player {player} selected pit {pit}")

    return game, turns, player1_turns, player2_turns




In [9]:
player1_wins = 0
player2_wins = 0
ties = 0

total_turns = 0
total_p1_turns = 0
total_p2_turns = 0


iterations = 0
start_time = time.perf_counter()

while iterations < 100:

    game, turns, player1_turns, player2_turns = play_game_alphabeta() #play_game_alphabeta()  #play_game_minmax()

    total_turns += turns

    total_p1_turns += player1_turns
    total_p2_turns += player2_turns

    

    

    player1_score = game.board[game.p1_mancala_index]
    player2_score = game.board[game.p2_mancala_index]

    if player1_score > player2_score:
        player1_wins += 1
    elif player1_score < player2_score: 
        player2_wins += 1
    else:
        ties += 1
    
    iterations += 1
    
    print("____________________________________________________")

end_time = time.perf_counter()
elapsed = end_time - start_time
print(f"Time taken: {elapsed:.4f} seconds")

player1_percent_win  = (player1_wins / iterations) * 100
player1_percent_loss = (player2_wins / iterations) * 100

player2_percent_win  = (player2_wins / iterations) * 100
player2_percent_loss = (player1_wins / iterations) * 100

tied_percent = (ties / iterations) * 100

average_number_of_turns = total_turns / iterations
average_number_of_turns_p1 = total_p1_turns / iterations
average_number_of_turns_p2 = total_p2_turns / iterations

print("Player 1 Stats")
print("_______________")
print("Games Won:", player1_percent_win, "%")
print("Games Lost:", player1_percent_loss, "%")

print("________________________________________________")

print("Player 2 Stats")
print("_______________")
print("Games Won:", player2_percent_win, "%")
print("Games Lost:", player2_percent_loss, "%")

print("________________________________________________")

print("Games Tied:", tied_percent, "%")

print("________________________________________________")

print("Average number of turns per game:", average_number_of_turns)
print("Average number of turns of player 1:", average_number_of_turns_p1)
print("Average number of turns of player 2:", average_number_of_turns_p2)


Player 1 chose pit: 6
Player 2 chose pit: 6
Player 1 chose pit: 1
Player 2 chose pit: 5
Player 1 chose pit: 2
Player 2 chose pit: 4
Player 1 chose pit: 4
Player 2 chose pit: 3
Player 1 chose pit: 5
Player 2 chose pit: 2
Player 1 chose pit: 2
Player 2 chose pit: 5
Player 1 chose pit: 2
Player 2 chose pit: 1
Player 1 chose pit: 3
Player 2 chose pit: 3
Player 1 chose pit: 5
Player 2 chose pit: 6
Player 1 chose pit: 1
Player 2 chose pit: 4
Player 1 chose pit: 6
Player 2 chose pit: 5
Player 1 chose pit: 2
Player 2 chose pit: 2
Player 1 chose pit: 3
Player 2 chose pit: 6
Player 1 chose pit: 1
Player 2 chose pit: 4
Player 1 chose pit: 4
Player 2 chose pit: 2
Player 1 chose pit: 6
Player 2 chose pit: 3
Player 1 chose pit: 5
Player 2 chose pit: 2
Player 1 chose pit: 2
Player 2 chose pit: 4
Player 1 chose pit: 3
Player 2 chose pit: 1
Player 1 chose pit: 4
Player 2 chose pit: 3
Player 1 chose pit: 6
Player 2 chose pit: 2
Player 1 chose pit: 5
Player 2 chose pit: 6
Player 1 chose pit: 2
Player 2 c